# LungHist700 Results Analysis

This notebook analyzes evaluation results from the LungHist700 dataset and generates summary tables.

## 🚀 New CLI Pipeline Feature

You can now generate these CSV summary tables automatically using the pipeline CLI:

```bash
# Generate CSV summary tables with the pipeline
python histobench/evaluation/lunghist700_pipeline.py --generate-summary

# Custom output directory
python histobench/evaluation/lunghist700_pipeline.py --generate-summary --summary-output-dir my_reports

# Or configure in config.yaml:
summary:
  generate: true
  output_dir: "reports/summary"
```

The pipeline will:
1. Run model evaluations 
2. Automatically generate summary tables in CSV format
3. Save them to the specified output directory
   - `lunghist700_knn_summary_TIMESTAMP.csv`
   - `lunghist700_linear_summary_TIMESTAMP.csv`

The CSV files are perfect for further processing, importing into Excel, or formatting as you prefer!

## Manual Analysis

The cells below show the manual process for generating these tables:

In [13]:
import pandas as pd
import glob
import os

In [14]:

# Directory containing your CV report CSVs
# report_dir = "/home/valentin/workspaces/histobench/reports/lunghist700"
report_dir = "/home/valentin/workspaces/histobench/reports/lunghist700_harder"
csv_files = glob.glob(os.path.join(report_dir, "*_cv_report*.csv"))

In [15]:

# Metrics to aggregate
metrics = ["accuracy", "precision", "recall", "f1"]

rows = []
for csv_path in csv_files:
    df = pd.read_csv(csv_path)
    # Extract method name from filename (model + aggregation)
    basename = os.path.basename(csv_path)
    method = basename.replace("_cv_report.csv", "")
    for clf in df["classifier"].unique():
        sub = df[df["classifier"] == clf]
        row = {"method": f"{method} ({clf})"}
        for m in metrics:
            if m in sub.columns:
                mean = sub[m].mean()
                std = sub[m].std()
                row[m] = f"{mean:.3f} ± {std:.3f}"
        rows.append(row)

summary = pd.DataFrame(rows)


In [16]:
# Split summary into KNN and Linear Classifier tables
summary_knn = summary[summary["method"].str.contains(r"\(knn\)", case=False)].reset_index(drop=True)
summary_linear = summary[summary["method"].str.contains(r"\(logistic_regression\)", case=False)].reset_index(drop=True)

summary_knn_sorted = summary_knn.sort_values(by="f1", ascending=False).reset_index(drop=True)
summary_linear_sorted = summary_linear.sort_values(by="f1", ascending=False).reset_index(drop=True)


# Display as markdown
print("### KNN Results")
display(summary_knn_sorted)
print("### Linear Classifier Results")
display(summary_linear_sorted)


### KNN Results


,method,accuracy,precision,recall,f1
0,UNI2_LungHist700_10x_whole_roi_KNNn_20 (knn),0.595 ± 0.096,0.560 ± 0.079,0.542 ± 0.065,0.506 ± 0.078
1,UNI2-20x_LungHist700_20x_whole_roi_KNNn_20 (knn),0.590 ± 0.098,0.555 ± 0.079,0.534 ± 0.065,0.498 ± 0.081
2,UNI2-20x_LungHist700_20x_tile_with_overlap_KNN...,0.573 ± 0.130,0.545 ± 0.081,0.530 ± 0.073,0.495 ± 0.101
3,UNI2_LungHist700_10x_tile_with_overlap_KNNn_20...,0.576 ± 0.116,0.527 ± 0.095,0.523 ± 0.077,0.490 ± 0.105
4,UNI2_LungHist700_10x_tile_with_overlap_KNNn_5 ...,0.542 ± 0.076,0.544 ± 0.057,0.505 ± 0.043,0.489 ± 0.060
5,UNI2-20x_LungHist700_20x_tile_with_overlap_KNN...,0.529 ± 0.031,0.522 ± 0.039,0.487 ± 0.025,0.469 ± 0.025
6,H-optimus-0_LungHist700_10x_tile_with_overlap_...,0.537 ± 0.111,0.509 ± 0.088,0.490 ± 0.083,0.460 ± 0.102
7,H-optimus-0_LungHist700_10x_tile_with_overlap_...,0.496 ± 0.058,0.485 ± 0.054,0.467 ± 0.047,0.448 ± 0.059
8,UNI2-20x_LungHist700_20x_whole_roi_KNNn_5 (knn),0.515 ± 0.067,0.483 ± 0.064,0.464 ± 0.056,0.442 ± 0.063
9,UNI2_LungHist700_10x_whole_roi_KNNn_5 (knn),0.513 ± 0.053,0.473 ± 0.040,0.459 ± 0.044,0.437 ± 0.046


### Linear Classifier Results


,method,accuracy,precision,recall,f1
0,UNI2_LungHist700_10x_tile_with_overlap_KNNn_5 ...,0.580 ± 0.093,0.553 ± 0.091,0.538 ± 0.098,0.522 ± 0.100
1,UNI2_LungHist700_10x_tile_with_overlap_KNNn_20...,0.580 ± 0.093,0.553 ± 0.091,0.538 ± 0.098,0.522 ± 0.100
2,UNI2_LungHist700_10x_whole_roi_KNNn_5 (logisti...,0.575 ± 0.104,0.553 ± 0.097,0.533 ± 0.100,0.515 ± 0.104
3,UNI2_LungHist700_10x_whole_roi_KNNn_20 (logist...,0.575 ± 0.104,0.553 ± 0.097,0.533 ± 0.100,0.515 ± 0.104
4,UNI2-20x_LungHist700_20x_tile_with_overlap_KNN...,0.576 ± 0.077,0.543 ± 0.095,0.529 ± 0.084,0.514 ± 0.095
5,UNI2-20x_LungHist700_20x_tile_with_overlap_KNN...,0.576 ± 0.077,0.543 ± 0.095,0.529 ± 0.084,0.514 ± 0.095
6,H-optimus-0_LungHist700_10x_tile_with_overlap_...,0.572 ± 0.106,0.538 ± 0.131,0.515 ± 0.133,0.511 ± 0.131
7,H-optimus-0_LungHist700_10x_tile_with_overlap_...,0.572 ± 0.106,0.538 ± 0.131,0.515 ± 0.133,0.511 ± 0.131
8,UNI2-20x_LungHist700_20x_whole_roi_KNNn_5 (log...,0.573 ± 0.107,0.550 ± 0.099,0.529 ± 0.105,0.511 ± 0.110
9,UNI2-20x_LungHist700_20x_whole_roi_KNNn_20 (lo...,0.573 ± 0.107,0.550 ± 0.099,0.529 ± 0.105,0.511 ± 0.110


In [10]:

# Generate LaTeX tables
latex_knn = summary_knn_sorted.to_latex(index=False, escape=False, column_format="l" + "c" * len(metrics))
latex_linear = summary_linear_sorted.to_latex(index=False, escape=False, column_format="l" + "c" * len(metrics))

print("\n% KNN Table\n", latex_knn)
print("\n% Linear Classifier Table\n", latex_linear)


% KNN Table
 \begin{tabular}{lcccc}
\toprule
method & accuracy & precision & recall & f1 \\
\midrule
UNI2_LungHist700_10x_tile_with_overlap_KNNn_5 (knn) & 0.565 ± 0.077 & 0.553 ± 0.055 & 0.532 ± 0.049 & 0.516 ± 0.052 \\
UNI2_LungHist700_10x_whole_roi_KNNn_20 (knn) & 0.585 ± 0.118 & 0.558 ± 0.070 & 0.537 ± 0.074 & 0.507 ± 0.091 \\
UNI2-20x_LungHist700_20x_whole_roi_KNNn_20 (knn) & 0.588 ± 0.112 & 0.561 ± 0.067 & 0.539 ± 0.067 & 0.506 ± 0.085 \\
UNI2-20x_LungHist700_20x_tile_with_overlap_KNNn_20 (knn) & 0.573 ± 0.107 & 0.548 ± 0.082 & 0.523 ± 0.075 & 0.500 ± 0.083 \\
UNI2_LungHist700_10x_whole_roi_KNNn_5 (knn) & 0.558 ± 0.063 & 0.535 ± 0.041 & 0.513 ± 0.047 & 0.499 ± 0.052 \\
UNI2-20x_LungHist700_20x_tile_with_overlap_KNNn_5 (knn) & 0.556 ± 0.057 & 0.546 ± 0.040 & 0.516 ± 0.028 & 0.499 ± 0.031 \\
UNI2-20x_LungHist700_20x_whole_roi_KNNn_5 (knn) & 0.552 ± 0.054 & 0.534 ± 0.038 & 0.507 ± 0.043 & 0.495 ± 0.049 \\
UNI2_LungHist700_10x_tile_with_overlap_KNNn_20 (knn) & 0.575 ± 0.119 & 0.548 ±

In [11]:

# Create copies with escaped method names for LaTeX
summary_knn_latex = summary_knn_sorted.copy()
summary_linear_latex = summary_linear_sorted.copy()

# Escape underscores in method names for LaTeX
summary_knn_latex['method'] = summary_knn_latex['method'].str.replace('_', r'\_')
summary_linear_latex['method'] = summary_linear_latex['method'].str.replace('_', r'\_')

# Generate LaTeX tables with escaped method names
latex_knn = summary_knn_latex.to_latex(index=False, escape=False, column_format="l" + "c" * len(metrics))
latex_linear = summary_linear_latex.to_latex(index=False, escape=False, column_format="l" + "c" * len(metrics))

print("\n% KNN Table\n", latex_knn)
print("\n% Linear Classifier Table\n", latex_linear)


% KNN Table
 \begin{tabular}{lcccc}
\toprule
method & accuracy & precision & recall & f1 \\
\midrule
UNI2\_LungHist700\_10x\_tile\_with\_overlap\_KNNn\_5 (knn) & 0.565 ± 0.077 & 0.553 ± 0.055 & 0.532 ± 0.049 & 0.516 ± 0.052 \\
UNI2\_LungHist700\_10x\_whole\_roi\_KNNn\_20 (knn) & 0.585 ± 0.118 & 0.558 ± 0.070 & 0.537 ± 0.074 & 0.507 ± 0.091 \\
UNI2-20x\_LungHist700\_20x\_whole\_roi\_KNNn\_20 (knn) & 0.588 ± 0.112 & 0.561 ± 0.067 & 0.539 ± 0.067 & 0.506 ± 0.085 \\
UNI2-20x\_LungHist700\_20x\_tile\_with\_overlap\_KNNn\_20 (knn) & 0.573 ± 0.107 & 0.548 ± 0.082 & 0.523 ± 0.075 & 0.500 ± 0.083 \\
UNI2\_LungHist700\_10x\_whole\_roi\_KNNn\_5 (knn) & 0.558 ± 0.063 & 0.535 ± 0.041 & 0.513 ± 0.047 & 0.499 ± 0.052 \\
UNI2-20x\_LungHist700\_20x\_tile\_with\_overlap\_KNNn\_5 (knn) & 0.556 ± 0.057 & 0.546 ± 0.040 & 0.516 ± 0.028 & 0.499 ± 0.031 \\
UNI2-20x\_LungHist700\_20x\_whole\_roi\_KNNn\_5 (knn) & 0.552 ± 0.054 & 0.534 ± 0.038 & 0.507 ± 0.043 & 0.495 ± 0.049 \\
UNI2\_LungHist700\_10x\_tile\_wit

In [12]:

def parse_method_name(method_name):
    """Parse long method names into shorter, more readable format."""
    import re
    
    # Extract base model name (everything before the first underscore after model name)
    base_match = re.match(r'^([^_]+(?:-[^_]+)*)', method_name)
    base_name = base_match.group(1) if base_match else method_name
    
    # Extract magnification (10x, 20x, etc.)
    mag_match = re.search(r'(\d+x)', method_name)
    magnification = mag_match.group(1) if mag_match else None
    
    # Extract k value for KNN
    k_match = re.search(r'KNNn?\_(\d+)', method_name)
    k_value = k_match.group(1) if k_match else None
    
    # Extract ROI type
    roi_type = None
    if 'whole_roi' in method_name:
        roi_type = 'wROI'
    elif 'tile_with_overlap' in method_name:
        roi_type = 'tROI'
    
    # Build the shortened name
    parts = [base_name]
    if magnification or k_value or roi_type:
        params = []
        if magnification:
            params.append(magnification)
        if roi_type:
            params.append(roi_type)
        if k_value:
            params.append(f"k={k_value}")
        parts.append(f"({', '.join(params)})")
    
    return ' '.join(parts)

# Create copies with parsed method names for LaTeX
summary_knn_latex = summary_knn_sorted.copy()
summary_linear_latex = summary_linear_sorted.copy()

# Parse and shorten method names, then escape underscores
summary_knn_latex['method'] = summary_knn_latex['method'].apply(parse_method_name).str.replace('_', r'\_')
summary_linear_latex['method'] = summary_linear_latex['method'].apply(parse_method_name).str.replace('_', r'\_')

# Generate LaTeX tables with parsed method names
latex_knn = summary_knn_latex.to_latex(index=False, escape=False, column_format="l" + "c" * len(metrics))
latex_linear = summary_linear_latex.to_latex(index=False, escape=False, column_format="l" + "c" * len(metrics))

print("\n% KNN Table\n", latex_knn)
print("\n% Linear Classifier Table\n", latex_linear)


% KNN Table
 \begin{tabular}{lcccc}
\toprule
method & accuracy & precision & recall & f1 \\
\midrule
UNI2 (10x, tROI, k=5) & 0.565 ± 0.077 & 0.553 ± 0.055 & 0.532 ± 0.049 & 0.516 ± 0.052 \\
UNI2 (10x, wROI, k=20) & 0.585 ± 0.118 & 0.558 ± 0.070 & 0.537 ± 0.074 & 0.507 ± 0.091 \\
UNI2-20x (20x, wROI, k=20) & 0.588 ± 0.112 & 0.561 ± 0.067 & 0.539 ± 0.067 & 0.506 ± 0.085 \\
UNI2-20x (20x, tROI, k=20) & 0.573 ± 0.107 & 0.548 ± 0.082 & 0.523 ± 0.075 & 0.500 ± 0.083 \\
UNI2 (10x, wROI, k=5) & 0.558 ± 0.063 & 0.535 ± 0.041 & 0.513 ± 0.047 & 0.499 ± 0.052 \\
UNI2-20x (20x, tROI, k=5) & 0.556 ± 0.057 & 0.546 ± 0.040 & 0.516 ± 0.028 & 0.499 ± 0.031 \\
UNI2-20x (20x, wROI, k=5) & 0.552 ± 0.054 & 0.534 ± 0.038 & 0.507 ± 0.043 & 0.495 ± 0.049 \\
UNI2 (10x, tROI, k=20) & 0.575 ± 0.119 & 0.548 ± 0.075 & 0.522 ± 0.077 & 0.494 ± 0.099 \\
H-optimus-0 (10x, tROI, k=5) & 0.502 ± 0.043 & 0.485 ± 0.054 & 0.461 ± 0.030 & 0.446 ± 0.034 \\
H-optimus-0 (10x, tROI, k=20) & 0.522 ± 0.098 & 0.481 ± 0.085 & 0.466 